In [6]:
# Let's make this notebook compatible for Python 2 and 3
from __future__ import division, print_function

# Import libraries
import pandas as pd
import numpy as np
import os
import math
import itertools
import string

# for visualization
import matplotlib.pyplot as plt

# to import module from parent directory
import sys
sys.path.append('..')

# Dataset API from sklearn
from sklearn import datasets

In [7]:
from utils import normalize, euclidean_distance, calculate_covariance_matrix, Plot

from deep_learning.optimizers import Adam
from deep_learning.loss_functions import CrossEntropy, SquareLoss
from deep_learning.layers import Dense, Dropout, Flatten, Activation, Reshape, BatchNormalization 
from deep_learning.neural_network import NeuralNetwork

In [10]:
class GeneticAlgorithm():
    """An implementation of a Genetic Algorithm which will try to produce the user
    specified target string.

    Parameters:
    -----------
    target_string: string
        The string which the GA should try to produce.
    population_size: int
        The number of individuals (possible solutions) in the population.
    mutation_rate: float
        The rate (or probability) of which the alleles (chars in this case) should be
        randomly changed.
    """
    def __init__(self, target_string, population_size, mutation_rate):
        self.target = target_string
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.letters = [" "] + list(string.ascii_letters)

    def _initialize(self):
        """ Initialize population with random strings """
        self.population = []
        for _ in range(self.population_size):
            # Select random letters as new individual
            individual = "".join(np.random.choice(self.letters, size=len(self.target)))
            self.population.append(individual)

    def _calculate_fitness(self):
        """ Calculates the fitness of each individual in the population """
        population_fitness = []
        for individual in self.population:
            # Calculate loss as the alphabetical distance between
            # the characters in the individual and the target string
            loss = 0
            for i in range(len(individual)):
                letter_i1 = self.letters.index(individual[i])
                letter_i2 = self.letters.index(self.target[i])
                loss += abs(letter_i1 - letter_i2)
            fitness = 1 / (loss + 1e-6)
            population_fitness.append(fitness)
        return population_fitness

    def _mutate(self, individual):
        """ Randomly change the individual's characters with probability
        self.mutation_rate """
        individual = list(individual)
        for j in range(len(individual)):
            # Make change with probability mutation_rate
            if np.random.random() < self.mutation_rate:
                individual[j] = np.random.choice(self.letters)
        # Return mutated individual as string
        return "".join(individual)

    def _crossover(self, parent1, parent2):
        """ Create children from parents by crossover """
        # Select random crossover point
        cross_i = np.random.randint(0, len(parent1))
        child1 = parent1[:cross_i] + parent2[cross_i:]
        child2 = parent2[:cross_i] + parent1[cross_i:]
        return child1, child2

    def run(self, iterations):
        # Initialize new population
        self._initialize()

        for epoch in range(iterations):
            population_fitness = self._calculate_fitness()

            fittest_individual = self.population[np.argmax(population_fitness)]
            highest_fitness = max(population_fitness)

            # If we have found individual which matches the target => Done
            if fittest_individual == self.target:
                break

            # Set the probability that the individual should be selected as a parent
            # proportionate to the individual's fitness.
            parent_probabilities = [fitness / sum(population_fitness) for fitness in population_fitness]

            # Determine the next generation
            new_population = []
            for i in np.arange(0, self.population_size, 2):
                # Select two parents randomly according to probabilities
                parent1, parent2 = np.random.choice(self.population, size=2, p=parent_probabilities, replace=False)
                # Perform crossover to produce offspring
                child1, child2 = self._crossover(parent1, parent2)
                # Save mutated offspring for next generation
                new_population += [self._mutate(child1), self._mutate(child2)]

            print ("[%d Closest Candidate: '%s', Fitness: %.2f]" % (epoch, fittest_individual, highest_fitness))
            self.population = new_population

        print ("[%d Answer: '%s']" % (epoch, fittest_individual))

In [11]:
def main():
    target_string = "Genetic Algorithm"
    population_size = 100
    mutation_rate = 0.05
    genetic_algorithm = GeneticAlgorithm(target_string,
                                        population_size,
                                        mutation_rate)

    print ("")
    print ("+--------+")
    print ("|   GA   |")
    print ("+--------+")
    print ("Description: Implementation of a Genetic Algorithm which aims to produce")
    print ("the user specified target string. This implementation calculates each")
    print ("candidate's fitness based on the alphabetical distance between the candidate")
    print ("and the target. A candidate is selected as a parent with probabilities proportional")
    print ("to the candidate's fitness. Reproduction is implemented as a single-point")
    print ("crossover between pairs of parents. Mutation is done by randomly assigning")
    print ("new characters with uniform probability.")
    print ("")
    print ("Parameters")
    print ("----------")
    print ("Target String: '%s'" % target_string)
    print ("Population Size: %d" % population_size)
    print ("Mutation Rate: %s" % mutation_rate)
    print ("")

    genetic_algorithm.run(iterations=1000)

if __name__ == "__main__":
    main()


+--------+
|   GA   |
+--------+
Description: Implementation of a Genetic Algorithm which aims to produce
the user specified target string. This implementation calculates each
candidate's fitness based on the alphabetical distance between the candidate
and the target. A candidate is selected as a parent with probabilities proportional
to the candidate's fitness. Reproduction is implemented as a single-point
crossover between pairs of parents. Mutation is done by randomly assigning
new characters with uniform probability.

Parameters
----------
Target String: 'Genetic Algorithm'
Population Size: 100
Mutation Rate: 0.05

[0 Closest Candidate: 'OjkBApedsxEfsximi', Fitness: 0.01]
[1 Closest Candidate: 'EjiVvcbpxBduDxLik', Fitness: 0.01]
[2 Closest Candidate: 'dxmhwznHvqcuupuAi', Fitness: 0.01]
[3 Closest Candidate: 'EjiVquRJvqcuupuAi', Fitness: 0.00]
[4 Closest Candidate: 'AweSRGdjHezhxgplh', Fitness: 0.01]
[5 Closest Candidate: 'HcTKojdjHezhxgplh', Fitness: 0.01]
[6 Closest Candidate: 'F

[138 Closest Candidate: 'Gcndvjdavjdokguin', Fitness: 0.03]
[139 Closest Candidate: 'Gcjdvjdavjdonitim', Fitness: 0.04]
[140 Closest Candidate: 'Gcjdvjdavjdonitim', Fitness: 0.04]
[141 Closest Candidate: 'Gcndvjdavjdosiujp', Fitness: 0.04]
[142 Closest Candidate: 'Gcndvjdavjdosiujp', Fitness: 0.04]
[143 Closest Candidate: 'Gcndvjbazjdonitim', Fitness: 0.05]
[144 Closest Candidate: 'Gcndvjbazjdonitin', Fitness: 0.05]
[145 Closest Candidate: 'Gcndvjbazjdoniuin', Fitness: 0.05]
[146 Closest Candidate: 'Genduodbzjdonitin', Fitness: 0.04]
[147 Closest Candidate: 'Gcndvjbazjdonitin', Fitness: 0.05]
[148 Closest Candidate: 'Ccndvjbazjdoniuin', Fitness: 0.04]
[149 Closest Candidate: 'Genduo azjdoniuin', Fitness: 0.04]
[150 Closest Candidate: 'Ccndvjbazjdoniuin', Fitness: 0.04]
[151 Closest Candidate: 'Genduo azjdotguin', Fitness: 0.04]
[152 Closest Candidate: 'CcndvjbaGldotguin', Fitness: 0.04]
[153 Closest Candidate: 'GcndvobaCjjonguin', Fitness: 0.03]
[154 Closest Candidate: 'GcndvjbaJjdotgu

[281 Closest Candidate: 'Gendtib Alhorithm', Fitness: 0.33]
[282 Closest Candidate: 'Gendtib Alhorithm', Fitness: 0.33]
[283 Closest Candidate: 'Gendtib Alhorithm', Fitness: 0.33]
[284 Closest Candidate: 'Gendtib Alhorithm', Fitness: 0.33]
[285 Closest Candidate: 'Gendtib Alhorithm', Fitness: 0.33]
[286 Closest Candidate: 'Gendtib Alhorithm', Fitness: 0.33]
[287 Closest Candidate: 'Gendtib Alhorithm', Fitness: 0.33]
[288 Closest Candidate: 'Gendtib Alhorithm', Fitness: 0.33]
[289 Closest Candidate: 'Gendtib Alhorithm', Fitness: 0.33]
[290 Closest Candidate: 'Gendtib Alhorithm', Fitness: 0.33]
[291 Closest Candidate: 'Gendtib Alhorithm', Fitness: 0.33]
[292 Closest Candidate: 'Gendtib Alhorithm', Fitness: 0.33]
[293 Closest Candidate: 'Gendtib Alhorithm', Fitness: 0.33]
[294 Closest Candidate: 'Gendtib Alhorithm', Fitness: 0.33]
[295 Closest Candidate: 'Genetib Alhorithm', Fitness: 0.50]
[296 Closest Candidate: 'Gendtib Alhorithm', Fitness: 0.33]
[297 Closest Candidate: 'Genetib Alhorit